# All or nothing pathfinders
1. Modify supply information in `sm.links` and `sm.road_links`
2. Run two possible all or nothing PT pathfinders:
    - one allowing walking on the road network
    - one allowing walking only on dedicated footpaths
3. Run on all or nothing road pathfinder

In [16]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [17]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
import os
import pandas as pd
if manual:
    %matplotlib inline

In [18]:
training_folder = '../../'

Load the model `base/walk_model.zip`.

# Additional supply data
- links length
- road_links time and walk_time

1. Compute the mean bus commercial speed from `sm.links` dataframe.
2. Set `road_links['time']` based on a `car_speed` 25% higher than bus commercial speed
3. Set `road_links['walk_time']` based on a walking speed three times slower than car.

# PT PATHFINDER
## walk on road
People walk on the road network, slow computation.  
Requires `'walk_time'` in `road_links` columns.

In [36]:
wor = sm.copy()
wor.step_pt_pathfinder(broken_modes=False, broken_routes=False, walk_on_road=True)

path_analysis: 100%|██████████████████████████████████████████████████████████| 12100/12100 [00:00<00:00, 16777.14it/s]


## as the crow flies
people walk on dedicated access and egress links only

In [37]:
acf = sm.copy()
acf.step_pt_pathfinder(broken_modes=False, broken_routes=False)

path_analysis: 100%|██████████████████████████████████████████████████████████| 12100/12100 [00:00<00:00, 16359.72it/s]


For all OD, compare gtime obtained with acf and obtained with wor.
1. by plotting both on a scatter plot
2. by comparing their statistics

In [38]:
# here we chose to use the ACF paths only
sm.pt_los = acf.pt_los

# ROAD PATHFINDER

In [39]:
sm.step_road_pathfinder(all_or_nothing=True)

100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [00:11<00:00,  9.77it/s]


In [11]:
if not os.path.isdir(training_folder + 'model/{scen}'.format(scen=scenario)):
    os.mkdir(training_folder + "model/%s" % scenario)
sm.to_zip(training_folder + 'model/{scen}/aon_pathfinder.zip'.format(scen=scenario))  

to_hdf(new file): 100%|████████████████████████████████████████████████████████████████| 38/38 [00:06<00:00,  5.44it/s]


# PLOT

Compare the PT and CAR level of service for all ODs.  

# END